In [1]:
from neural_network.binary_nn import NN
from datasets.sipm_dataset import SiPMDataset
from genetic_algorithm.algorithm import GeneticAlgorithm

In [2]:
# GA parameters
mutation_prob = 0.05
pop_size = 10       # small population for demo
ngen = 5            # few generations
n_samples = 128

# nn = NN((genome_length, 64, 128, 2), input=SiPMDataset(n_samples=genome_length))
nn = NN((n_samples, 16, 128, 2), input=SiPMDataset(n_samples=n_samples))

ga = GeneticAlgorithm(
    nn=nn, 
    pop_size=pop_size,
    ngen=ngen,
    elite_size=2,
)

TypeError: GeneticAlgorithm.__init__() got an unexpected keyword argument 'genome_length'

In [ ]:
pop, log, hof = ga.run()

Create init population...
Start evolution...
   	      	        divers        	                          fitness                           	         size         
   	      	----------------------	------------------------------------------------------------	----------------------
gen	nevals	gen	max	nevals	avg	diversity	gen	max	min	nevals	time	gen	max  	nevals
0  	10    	0  	0  	10    	0  	0.1      	0  	0  	0  	10    	8 s 	0  	0.105	10    
1  	10    	1  	0  	10    	0  	0.1      	1  	0  	0  	10    	19 s	1  	0.11 	10    
2  	6     	2  	0  	6     	0  	0.1      	2  	0  	0  	6     	26 s	2  	0.11 	6     
3  	10    	3  	0  	10    	0  	0.1      	3  	0  	0  	10    	38 s	3  	0.11 	10    
4  	10    	4  	0  	10    	0  	0.1      	4  	0  	0  	10    	50 s	4  	0.115	10    
5  	8     	5  	0  	8     	0  	0.1      	5  	0  	0  	8     	59 s	5  	0.115	8     
Evolution finished.


In [4]:
print("Best individual:", hof[0])
print("Best fitness:", ga.nn.fitness(hof[0]))
print("GA run complete.\n")

Best individual: [np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int6

# Testing

In [5]:
nn = NN((2, 8, 2), input=SiPMDataset())
x, y = SiPMDataset().load_data()
print(f"Inputs (X): \n{x}\n\n\nLabels (y): \n{y}\n\n\n")

Inputs (X): 
[[ 8  8 40 ...  8  8  8]
 [ 8  8 42 ...  8  8  8]
 [ 8  8 42 ...  8  8  8]
 ...
 [ 8  8 10 ...  8  8  8]
 [ 8  8 16 ...  8  8  8]
 [ 8  8 14 ...  8  8  8]]


Labels (y): 
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]



